# Description
Let's see what properties we can find :) <br>
Let's put more data into it !

## TODOs
 - scale frequency spectrum before
 - species level precision (instead of strand level)
 - automate all : 
  - PCA/LDA/Autoencoder/nothing
  - number of bacteria and samples
 - treat plasmids differently
 - try with real Nanopore reads
 - unsupervised learning on all windows into 10 bins ?
 - hierarchical clustering ??
 - clustering with minimal distance
 - taxo id to family or genus
 - tsne plot
 - Add **Visualisation** : https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html
 - Save each case, not each run. don't run cases that have already been computed.

## Structure
In `genome/`, there's multiple sub-folder, we will start with `Bacteria`
It then contains all recorded species/strands in individual folders


http://defindit.com/readme_files/ncbi_file_extension_format.html

What we need is the taxo id, name, and the DNA, which can be found in:
 - .gbk for the taxo and name
 - .fna for the sequence

# Import and Paths

In [1]:
print("let's start")

let's start


In [15]:
import os
from os import path as osp
import pandas as pd
import numpy as np
import pickle
# import traceback
import multiprocessing
import random
import subprocess
# from collections import Counter
from datetime import datetime as dt
from time import time
from tqdm import tqdm_notebook as tqdm

from Bio import SeqIO, SeqRecord

In [4]:
# from sklearn.preprocessing import MinMaxScaler, scale
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [5]:
plt.rcParams['figure.figsize'] = 13, 8

In [6]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
path_ref_db = "/home/ubuntu/Data/NCBI/Bacteria_2015/"
path_kmer_freq = "/home/ubuntu/Data/kmer_freq/"

In [ ]:
path_4mer = "4mer/V4/"
path_4mer = os.path.join(path_kmer_freq, path_4mer)

In [ ]:
path_all = os.path.join(path_4mer, "_all_bacteria_4mers.largepd")
print(f"{os.path.isfile(path_all)} path_all")

# Functions

In [10]:
nucleotides = "ACGT"

In [11]:
def kmers_dic(n, choice=nucleotides):
    return {a:0 for a in combinaisons(choice, n)}

In [13]:
def combinaisons(combi, n, instances=nucleotides):
    if n == 1:
        return combi
    else:
        return [f"{a}{n}" for a in combinaisons(combi, n-1) for n in instances]

## Unused methods

In [ ]:
def read_fna(file_path):
    with open(file_path) as f:
        rec = f.readlines()
        return "".join(rec[1:]).replace("\n", "")

In [ ]:
def window(fseq, window_size=4):
    for i in range(len(fseq) - window_size + 1):
        yield fseq[i:i+window_size]

In [ ]:
def count_all_kmers(seq, kmer_template, n, ignore_N=True):
    """ Count all kmers, ignore kmers with N or other undecided nucleotides 
        Return a list of dict for each window (w=100)
        seq: string nucleotide input
        kmer_template: dict with all combinations of nucleotides, initialized with zeros (kmer_template["AAAA"] = 0, kmer_template["AAAC"] = 0...)
    """
    kmer_counts = kmer_template.copy()
    wrong_base = "N"*n
    kmer_counts[wrong_base] = 0
    
    try:
        for i, kmer in enumerate(window(seq, n)):
            try:
                kmer_counts[kmer] += 1
            except:
                kmer_counts[wrong_base] += 1
                
        if ignore_N:
            kmer_counts.pop(wrong_base)
        return kmer_counts
    except Exception as e:
        print("type error: " + str(e))
        print(traceback.format_exc())
        return kmer_counts

In [ ]:
def count_kmers(seq, kmer_template, n, bacteria_name, fna, w=100):
    """ Count all kmers, ignore kmers with N or other undecided nucleotides 
        Return a list of dict for each window (w=100)
    """
    res = []
    current_split = 0
    next_split = current_split + w
    tmp_counts = kmer_template.copy()
    tmp_counts["start"] = current_split
    
    try:
        for i, kmer in enumerate(window(seq, n)):
            try:
                tmp_counts[kmer] += 1
            except:
                pass
            # To lower the computational need to split into windows
            if i == next_split:
                res.append(tmp_counts)
                current_split = next_split
                next_split += w
                tmp_counts = kmer_template.copy()
                tmp_counts["start"] = current_split
                
        return i+1, res
    except Exception as e:
        print("type error: " + str(e))
        print(traceback.format_exc())
        return i, res

# Classes

In [16]:
class CustomRead(SeqRecord.SeqRecord):
    KMER4 = kmers_dic(4)
    FASTQ_PATH = None
    BASE_PATH  = None
    
    # Load the models to be able to apply them on each read
    _models = "/home/ubuntu/Data/kmer_freq/4mer/V4"
    _file_lda_model = f"{_models}/LDA/lda_model_20_int.pd"
    _file_kmeans_model = f"{_models}/clustering/10_kmeans_2019-05-09_04-08.pkl"
    LDA = pickle.load(open(_file_lda_model, 'rb'))
    KMEANS = pickle.load(open(_file_kmeans_model, 'rb'))
    
    def __init__(self, obj, k=4):
        # wrap the object
        self._wrapped_obj = obj
        # Additional attributes
        self.k          = k
        self.bin        = None
        self.kmer_count = self.KMER4.copy()
        self.lda_feat   = []
        self.path_out   = None
    
    def __getattr__(self, attr):
        if attr in self.__dict__:
            return getattr(self, attr)
        return getattr(self._wrapped_obj, attr)
    
    def count_kmer(self):
        """"the new string hashing behaviour, BiopythonWarning: Using str(self.seq) to use the new behaviour"""
        self.kmer_count = count_all_kmers(str(self.seq), self.kmer_count, self.k)
    
    def lda_reduce(self):
        self.lda_feat   = self.LDA.transform(np.fromiter(
                          self.kmer_count.values(),dtype=int).reshape(-1, 256))  # Put into 2D one row
        
    def find_bin(self):
        self.bin        = self.KMEANS.predict(self.lda_feat)[0]
        self.description= self.description + f", bin_id={self.bin}"
        self.path_out   = f"{self.BASE_PATH}.bin-{self.bin}.fastq"
        return self.bin
    
    def to_fastq(self):
        assert self.FASTQ_PATH is not None, AttributeError("Path of the fastq file must first be defined")
        with open(self.path_out, "a") as f:
            SeqIO.write(self, f, "fasta")
        
    def set_fastq_path(path_fastq):
        assert osp.isfile(path_fastq), FileNotFoundError(f"{path_fastq} cannot be found")
        CustomRead.FASTQ_PATH = path_fastq
        CustomRead.BASE_PATH  = osp.splitext(path_fastq)[0]

In [17]:
class FastQClassification:
    _path_database = "/home/ubuntu/database/kraken2"
    KRAKEN2_DB = {
        "2015-10bins"  : f"{_path_database}/2015-10bins",
        "2015-standard": f"{_path_database}/2015-standard",
    }
    FOLDER_REPORTS = "/home/ubuntu/Data/Reports"
    
    def __init__(self, path_original_fastq, db_choice, bin_nb=10, classifier="kraken2",
                 cores=multiprocessing.cpu_count(), folder_report="/home/ubuntu/Data/Reports", 
                 dry_run=True, verbose=False):
        assert osp.isfile(path_original_fastq), FileNotFoundError(f"Didn't find original fastq {path_original_fastq}")
        
        self.path_original_fastq    = path_original_fastq
        self.folder, self.file_name = osp.split(osp.splitext(self.path_original_fastq)[0])
        self.path_binned_fastq      = []              # [(<bin i>, <path_file>), ]
        self.folder_report          = folder_report
        
        self.classifier    = classifier
        self.db_choice     = db_choice  # Either full or bins: 2015-standard / 2015-bins10
        self.bin_nb        = bin_nb
        self.folder_out    = f"{self.folder_report}/{self.file_name}"
        if not os.path.isdir(self.folder_out):
            os.makedirs(self.folder_out)
        self.path_out      = f"{self.folder_out}/{self.db_choice}"
        
        self.cores         = cores
        self.dry_run       = dry_run
        self.verbose       = verbose
        self.cmd           = None
    
                
    def find_binned_files(self):
        for i in range(self.bin_nb):
            path_bin_i = f"{self.folder}/{self.file_name}.bin-{i}.fastq"
            if osp.isfile(path_bin_i): 
                self.path_binned_fastq.append((i, path_bin_i))
    
    def classify(self):
        if self.classifier == "kraken2":
            if "bins" in self.db_choice:
                self.find_binned_files()
                for i, file in tqdm(self.path_binned_fastq):
                    print(f"bin {i} db loading... {file}")
                    self.kraken2_bins(i, file)
            else:
                print("standard full db loading...")
                self.kraken2_standard()
                
    def kraken2_standard(self):
        self.cmd = [
            "kraken2", "--threads", f"{self.cores}",
            "--db", f"{self.KRAKEN2_DB[self.db_choice]}", 
            self.path_original_fastq, 
            "--output", f"{self.path_out}.full.kraken2.out",
            "--report", f"{self.path_out}.full.kraken2.report",
        ]
        if self.verbose: print(" ".join(self.cmd))
        if not self.dry_run:            
            results = subprocess.check_output(self.cmd)
            if self.verbose: print(results)
                
    def kraken2_bins(self, i, file):
        self.cmd = [
            "kraken2", "--threads", f"{self.cores}",
            "--db", f"{self.KRAKEN2_DB[self.db_choice]}/{i}", 
            file, 
            "--output", f"{self.path_out}.bin-{i}.kraken2.out",
            "--report", f"{self.path_out}.bin-{i}.kraken2.report",
        ]
        if self.verbose: print(" ".join(self.cmd))
        if not self.dry_run:            
            results = subprocess.check_output(self.cmd)
            if self.verbose: print(results)
            
    def kraken2_report_merging(self):
        NotImplementedError
        
    
    def __repr__(self):
        return f"Fastq file located at <{self.path_original_fastq}>, ready to be classified with " \
               f"{self.classifier} with the DB <{self.db_choice}>"
        

# Run Kraken2 on each bin

In [19]:
path_fastq_comm = "/home/ubuntu/Disks/HDD1000/Segmentation/Test-Data/Synthetic_from_Genomes/2019-12-05_100000-WindowReads_20-BacGut/2019-12-05_100000-WindowReads_20-BacGut.fastq"

In [20]:
fastq_classification = FastQClassification(path_fastq_comm, "2015-10bins")

In [25]:
fastq_classification.dry_run=False
fastq_classification.verbose=False

In [22]:
fastq_classification.db_choice = "2015-10bins"

In [24]:
fastq_classification.classify()

bin 1 db loading... /home/ubuntu/Disks/HDD1000/Segmentation/Test-Data/Synthetic_from_Genomes/2019-12-05_100000-WindowReads_20-BacGut/2019-12-05_100000-WindowReads_20-BacGut.bin-1.fastq
kraken2 --threads 2 --db /home/ubuntu/database/kraken2/2015-10bins/1 /home/ubuntu/Disks/HDD1000/Segmentation/Test-Data/Synthetic_from_Genomes/2019-12-05_100000-WindowReads_20-BacGut/2019-12-05_100000-WindowReads_20-BacGut.bin-1.fastq --output /home/ubuntu/Data/Reports/2019-12-05_100000-WindowReads_20-BacGut/2015-10bins.bin-1.kraken2.out --report /home/ubuntu/Data/Reports/2019-12-05_100000-WindowReads_20-BacGut/2015-10bins.bin-1.kraken2.report


KeyboardInterrupt: 

In [23]:
fastq_classification.db_choice = "2015-standard"

In [25]:
fastq_classification.classify()

standard


End of latest scripts

# Create reads

## Synthetic reads from the Ref Database
find reads, real or from ncbi
only genome, not plasmid ?

In [19]:
nb_reads = 10000
seed = 34
nb_species = len(os.listdir(path_ref_db))

In [16]:
def reads_from_ncbi_genomes():
    fake_reads = {"bac": [], "seq": []}
    nb_species = len(os.listdir(path_ref_db))
    indices = sorted(np.random.choice(range(nb_species), nb_reads))
    indices_multi = Counter(indices)
    
    for i, file_name in tqdm(enumerate(os.listdir(path_ref_db)), total=nb_species):
        if i in indices_multi.keys():
    #         print(i, indices_multi[i], file_name)
            try:
                fake_reads["bac"].extend([file_name]*indices_multi[i])
                folder_bacteria = os.path.join(path_ref_db, file_name)
                all_fna = sorted([file.path for file in os.scandir(folder_bacteria) if file.name.endswith(".fna")])
                all_fna_sizes = [os.path.getsize(f) for f in all_fna]
                max_value = max(all_fna_sizes)
                max_index = all_fna_sizes.index(max_value)
                rec = read_fna(all_fna[max_index]) 

                rec_len = len(rec)
                reads_len = [random.randint(1000, min(20000, rec_len)) for k in range(indices_multi[i])]
                reads_start = [random.randint(0, rec_len-reads_len[k]) for k in range(indices_multi[i])]

                for j in range(indices_multi[i]):
                    fake_reads["seq"].append(rec[reads_start[j] : reads_start[j]+reads_len[j]])
    #                 print(reads_start[j], reads_start[j]+reads_len[j], rec_len, len(fake_reads["seq"][-1]))
            except Exception as e:
                print(folder_bacteria)
                print(e)
                
    return fake_reads

In [37]:
fake_reads = reads_from_ncbi_genomes()

In [38]:
save_fake_reads = fake_reads.copy()

In [61]:
fake_reads = save_fake_reads.copy()

# kmer count
All random sequences from NCBI have been generated <br>
Now, count their kmer frequencies

In [19]:
k = 4
kmer4 = kmers_dic(k)
cols_kmers = list(kmer4.keys())

## Obsolete methods

In [65]:
for key in kmer4.keys():
    fake_reads[key] = []

Count and add counts besides the Dataframe

In [68]:
for i in tqdm(range(len(fake_reads["bac"]))):
    counts = count_all_kmers(fake_reads["seq"][i], kmer4, k)
    for key in kmer4.keys():
        fake_reads[key].append(counts[key])

In [89]:
len(fake_reads["seq"]), len(fake_reads["bac"]), len(fake_reads["AAAA"])

(10000, 10000, 10000)

In [70]:
pd_reads = pd.DataFrame(fake_reads)

In [71]:
pd_reads.head(5)

,bac,seq,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,Acaryochloris_marina_MBIC11017_uid58167,AACGATGAAAATTTTTAGCTTTATTTAGAGGTTGATATCAAAAATA...,99,101,81,104,53,98,64,69,...,55,77,121,125,95,63,66,92,130,113
1,Acaryochloris_marina_MBIC11017_uid58167,TGGCAAAATCGTCGGAATTTCTGGTTCCCTGGCGGACGTAACCGAA...,211,104,119,189,114,86,44,93,...,63,117,129,95,121,89,136,124,134,169
2,Acaryochloris_marina_MBIC11017_uid58167,GCAGGGGAAACTGCCGTAAATAATTGTCTTTGGTCGTTAAAGGTAG...,165,78,131,161,81,76,63,87,...,69,126,169,117,133,87,109,89,143,144
3,Acetobacterium_woodii_DSM_1030_uid88073,ATGTGTCGTTGCTTGTCCGGTTGGGAAGAAGGCAACCGCCCCCAAA...,74,28,32,49,15,21,15,17,...,7,13,29,15,18,25,43,15,20,49
4,Acetobacterium_woodii_DSM_1030_uid88073,TAATGCCTCAATTGTTTTAAGTAAGTTCTGGTAGGAAACCTGAAAA...,58,41,37,66,30,51,27,18,...,39,55,43,32,29,29,63,103,50,135


In [72]:
pd_reads.shape

(10000, 258)

## Open fastq file, load each read, count its kmers, bin it

In [22]:
counter = 0
read_to_bin = []
CustomRead.set_fastq_path(path_fastq_comm)

for record in tqdm(SeqIO.parse(path_fastq_comm, "fasta")):
    custom_read = CustomRead(record)
    custom_read.count_kmer()
    custom_read.lda_reduce()
    custom_read.find_bin()
    custom_read.to_fastq()
    
    counter += 1
#     if counter > 1000:
#         break
print(counter)


100000


# Reload and apply the LDA model

LDA trained on reference genomes

In [73]:
file_lda_model = f"/home/ubuntu/Data/kmer_freq/4mer/V4/LDA/lda_model_20_int.pd"
lda = pickle.load(open(file_lda_model, 'rb'))

In [74]:
components = 20
np_lda = lda.transform(pd_reads[cols_kmers])
cols_lda = [f"lda_{i+1}" for i in range(components)]
df_lda = pd.DataFrame(np_lda, columns=cols_lda)
df_lda = pd.concat([pd_reads["bac"].reset_index(inplace=False, drop=True), df_lda], axis=1) # , ignore_index=True)

In [75]:
df_lda.head(5)

,bac,lda_1,lda_2,lda_3,lda_4,lda_5,lda_6,lda_7,lda_8,lda_9,...,lda_11,lda_12,lda_13,lda_14,lda_15,lda_16,lda_17,lda_18,lda_19,lda_20
0,Acaryochloris_marina_MBIC11017_uid58167,2.16,-0.28,5.55,-10.76,4.89,-6.12,2.53,7.11,-0.47,...,-1.51,1.14,-6.35,-4.26,2.56,1.34,-0.16,4.96,-9.48,-1.58
1,Acaryochloris_marina_MBIC11017_uid58167,7.69,1.91,9.66,-8.39,6.02,-3.14,-0.82,5.56,-5.09,...,-1.16,-2.88,-3.52,-6.08,2.89,-2.78,4.44,7.40,-8.57,-0.81
2,Acaryochloris_marina_MBIC11017_uid58167,5.18,1.26,11.39,-10.15,5.89,-4.16,-0.13,6.67,-2.31,...,-0.70,-2.40,-3.01,-5.69,4.04,-1.07,1.68,6.49,-7.73,-3.51
3,Acetobacterium_woodii_DSM_1030_uid88073,1.29,-0.41,-3.68,2.16,-1.18,-2.08,-1.61,-0.32,0.41,...,0.51,2.72,1.40,3.36,-2.25,1.26,0.82,-4.36,4.00,-0.97
4,Acetobacterium_woodii_DSM_1030_uid88073,4.01,-1.99,-1.75,-0.49,-1.00,-3.18,-2.52,-0.71,0.82,...,0.93,1.75,-0.82,1.49,-0.19,-1.04,0.92,-0.27,-1.69,-0.39


## Scaling

In [76]:
df_lda[cols_lda] = scale(df_lda[cols_lda])

# Load classifier and bin reads

In [77]:
file_NB_model = f"/home/ubuntu/Data/kmer_freq/4mer/V4/ml_models/_2782_Naive_Bayes_2019-05-08_17-24.pkl"
naive_bayes = pickle.load(open(file_NB_model, 'rb'))

In [78]:
pred = naive_bayes.predict(df_lda[cols_lda])

In [79]:
df_lda["pred"] = pred

In [80]:
df_lda["genus_match"] = (df_lda.bac.str.split("_", 1).str[0] ==
                         df_lda[f"pred"].str.split("_", 1).str[0]).astype(int)

In [86]:
matches = df_lda[["genus_match"]].sum()
print(f"{matches[0]} matches, {100*matches[0]/df_lda.shape[0]:.1f}%")

4951 matches, 49.5%


# Binning by clustering

In [91]:
with open(f"/home/ubuntu/Data/kmer_freq/4mer/V4/clustering/10_agglo_2019-05-09_04-08.pkl", 'rb') as f: 
    agglo = pickle.load(f)

In [92]:
with open(f"/home/ubuntu/Data/kmer_freq/4mer/V4/clustering/10_kmeans_2019-05-09_04-08.pkl", 'rb') as f: 
    kmeans = pickle.load(f)

# End of the script.
Sylvain @GIS